# Simulating Data Poisoning Attacks against Twitter Recommender

This tutorial details experiments designed to simulate attacks against social network recommendation mechanisms, and evaluate their effectiveness. All experiments documented here follow the procedure outlined below:

1. Load a dataset of anonymized retweet interactions collected from actual Twitter data.
2. Train a collaborative filtering model on the loaded data.
3. Select a target account to be "amplified" such that it is recommended to a set of users who have interacted with a separate, high-profile user also in the dataset. We select 20 such users as a "control" set.
4. Implement recommendation logic based on cosine similarity of the vector representations of the trained model, and observe recommendations for the control set.
5. Select a set of "amplifier accounts" that have not interacted with either the target account or the high-profile account, and are not members of the control set.
6. For a number of different proposed sets of amplifier accounts and parameter choices, create a new dataset containing additional interactions between each selected amplifier account and both the target account and the high-profile account. In practise, this process involves appending two new rows per amplifier account - one adding a retweet count for the target account and another adding retweet count for the high-profile account.
7. Train a new model on the modified dataset.
8. Run both target-based and source-based recommendations for each member of the control group and record the number of times the target appeared in the top-n (3) recommendations.
9. Present and discuss the results.

In [ ]:
!pip install -U fastai

In [ ]:
# Note this is broken for pytorch 1.7.x, so please use pytorch 1.6
# pip install torch==1.6.0 torchvision==0.7.0

from sklearn.metrics.pairwise import cosine_similarity

from fastai.tabular.all import *
from fastai.collab import *

import networkx as nx
import community
import community.community_louvain as community_louvain

import pandas as pd
import numpy as np
import json
import os

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

random.seed(1)

# Helper functions
def save_ratings(df, fn):
    with open(fn, "w") as f:
        f.write("Source,Target,Weight\n")
        for item in zip(df["Source"], df["Target"], df["Weight"]):
            s, t, w = item
            f.write(str(s)+","+str(t)+","+str(w)+"\n")

def key_with_max_value(d):  
     v = list(d.values())
     k = list(d.keys())
     return k[v.index(max(v))], max(v)

def key_with_min_value(d):  
     v = list(d.values())
     k = list(d.keys())
     return k[v.index(min(v))], min(v)

def get_most_similar(uid, matrix, max_matches):
    if uid >= len(matrix):
        return None
    all_matches = matrix[uid]
    top_matches = np.flip(np.argsort(all_matches))
    match_rating = [[top_matches[i], all_matches[top_matches[i]]] for i in range(max_matches)]
    return match_rating

def print_similar_to_targets(samples, t_matrix):
    for n in samples:
        print("Target: " + tid_name[n] + " similar to:")
        matches = get_most_similar(n, t_matrix, t_max_matches)
        if matches == None:
            return
        for item in matches[:10]:
            tid, rating = item
            print(tid_name[tid] + " " + "%.4f"%rating)
        print()

def print_similar_to_sources(samples, s_matrix):
    for n in samples:
        print("User: " + sid_name[n] + " similar to:")
        matches = get_most_similar(n, s_matrix, s_max_matches)
        if matches == None:
            return
        for item in matches[:10]:
            sid, rating = item
            print(sid_name[sid] + " " + "%.4f"%rating)
        print()
        
def print_target_similarity(t1, t2, t_matrix):
    sim = t_matrix[t1][t2]
    print(tid_name[t1] + " similarity to " + tid_name[t2] + ": " + "%.4f"%sim)

def make_nx_graph(inter):
    mapping = []
    names = set()
    for source, targets in inter.items():
        names.add(source)
        for target, count in targets.items():
            mapping.append((source, target, count))
            names.add(target)
    g=nx.Graph()
    g.add_weighted_edges_from(mapping)
    return g, names

def get_median(var, source_list):
    vals = []
    for s in source_list:
        if s in var:
            vals.append(var[s])
    return np.median(vals)

def get_mean(var, source_list):
    vals = []
    for s in source_list:
        if s in var:
            vals.append(var[s])
    return np.mean(vals)

def get_jaccard_median(inter, source_list, target):
    g, names = make_nx_graph(inter)
    pairs = []
    for s in source_list:
        if s in names:
            pairs.append((s, target))
    preds = nx.jaccard_coefficient(g, pairs)
    vals = []
    for s, t, p in preds:
        vals.append(p)
    return np.median(vals)

def get_jaccard_mean(inter, source_list, target):
    g, names = make_nx_graph(inter)
    pairs = []
    for s in source_list:
        if s in names:
            pairs.append((s, target))
    preds = nx.jaccard_coefficient(g, pairs)
    vals = []
    for s, t, p in preds:
        vals.append(p)
    return np.mean(vals)

def get_communities(inter):
    g, names = make_nx_graph(inter)
    communities = community_louvain.best_partition(g)

    clusters = {}
    for node, mod in communities.items():
        if mod not in clusters:
            clusters[mod] = []
        clusters[mod].append(node)
    return clusters

def get_mean_distance(inter, target, source_list):
    g, names = make_nx_graph(inter)

    distance_vals = []
    for source in source_list:
        if source in names:
            length = nx.shortest_path_length(g, source=target, target=source)
            distance_vals.append(length)
    return np.mean(distance_vals)

# Return a new poisoned dataframe
def get_poisoned_dataset(ratings, amplifier_candidates, num_amplifiers, rating_val, save_path):
    ratings2 = pd.DataFrame(ratings)
    # For base set measurements
    if num_amplifiers < 1 or rating_val < 1:
        return ratings2
    new_data = []
    amplifiers = random.sample(amplifier_candidates, num_amplifiers)

    for uid in amplifiers:
        new_data.append([uid, target_tid, rating_val])
        new_data.append([uid, high_profile_tid, rating_val])
    new_ratings_df = pd.DataFrame(new_data, columns=['Source', 'Target', 'Weight'])
    ratings2 = ratings2.append(new_ratings_df, ignore_index=True)
    
    # Save poisoned dataset for further inspection or visualization in gephi
    interactions2 = {}
    for item in zip(ratings2['Source'], ratings2['Target'], ratings2['Weight']):
        s, t, r = item
        sid_label = sid_name[s]
        tid_label = tid_name[t]
        if sid_label not in interactions2:
            interactions2[sid_label] = Counter()
        interactions2[sid_label][tid_label] += r
    with open(save_path, "w") as f:
        f.write("Source,Target,Weight\n")
        for s, tw in interactions2.items():
            for t, w in tw.items():
                f.write(str(s)+","+str(t)+","+str(w)+"\n")
    return ratings2

## Load and process dataset

In [ ]:
!wget -q --show-progress https://github.com/r0zetta/collaborative_filtering/raw/main/US2020/anonymized_interactions.csv

anonymized_interact 100%[===================>]   1.31M  --.-KB/s    in 0.02s   


In [ ]:
# Load and prepare raw data
raw = pd.read_csv("anonymized_interactions.csv")

# Source ids (accounts that retweeted)
sid_name = {}
name_sid = {}
sid = 0
for name in raw['Source']:
    if name not in name_sid:
        name_sid[name] = sid
        sid_name[sid] = name
        sid += 1

# Target ids (accounts that received retweets)
tid_name = {}
name_tid = {}
tid = 0
for name in raw['Target']:
    if name not in name_tid:
        name_tid[name] = tid
        tid_name[tid] = name
        tid += 1

print("Number of retweeters: " + str(len(name_sid)))
print("Number of retweeted: " + str(len(name_tid)))
# Assemble ratings dataframe used to train the model
ratings = pd.DataFrame()
ratings['Source'] = [name_sid[x] for x in raw['Source']]
ratings['Target'] = [name_tid[x] for x in raw['Target']]
ratings['Weight'] = raw['Weight']
ratings.head()

Number of retweeters: 25137
Number of retweeted: 8405


,Source,Target,Weight
0,0,0,1
1,0,1,4
2,0,2,1
3,0,3,1
4,0,4,1


In [ ]:
source_list = list(set(ratings['Source']))
target_list = list(set(ratings['Target']))
target_retweeted_by = {}
target_retweeted_count = {}
target_retweeters = Counter()
target_source_count = Counter()
source_retweeted = {}
source_retweets = Counter()
source_target_count = Counter()
interactions = {}
for item in zip(ratings['Source'], ratings['Target'], ratings['Weight']):
    s, t, r = item
    if sid_name[s] not in interactions:
        interactions[sid_name[s]] = Counter()
    interactions[sid_name[s]][tid_name[t]] += r
    source_retweets[s] += r
    if t not in target_retweeted_count:
        target_retweeted_count[t] = Counter()
    target_retweeted_count[t][s] = r
    if s not in source_retweeted:
        source_retweeted[s] = []
    if t not in source_retweeted[s]:
        source_retweeted[s].append(t)
        source_target_count[s] += 1
    if t not in target_retweeted_by:
        target_retweeted_by[t] = []
    if s not in target_retweeted_by[t]:
        target_retweeted_by[t].append(s)
        target_source_count[t] += 1
    target_retweeters[t] += 1
with open("labeled_ratings.csv", "w") as f:
    f.write("Source,Target,Weight\n")
    for s, tw in interactions.items():
        for t, w in tw.items():
            f.write(str(s)+","+str(t)+","+str(w)+"\n")
print("Number of sources: " + str(len(source_list)))
print("Number of targets: " + str(len(target_list)))
print("Total number of retweet interactions: " + str(sum(ratings['Weight'])))
print()
print("Targets with most retweets")
print("tid\tretweets")
for x, c in target_retweeters.most_common(10):
    print(tid_name[x] + "\t" + str(c))
print()
print("Targets with most unique sources retweeting them")
print("tid\tsources")
for x, c in target_source_count.most_common(10):
    print(tid_name[x] + "\t" + str(c))
print()
for x, c in target_retweeters.most_common(10):
    also_retweeted = Counter()
    for sid, tids in source_retweeted.items():
        if len(tids) > 1:
            if x in tids:
                for tid in tids:
                    if tid != x:
                        also_retweeted[tid] += 1
    msg = "Sources that retweeted " + tid_name[x]
    msg += " also retweeted " + str(len(also_retweeted)) + " other accounts."
    print(msg)
    for x, c in also_retweeted.most_common(10):
        print("Retweeted " + tid_name[x] + " " + str(c) + " times.")
    print("")

# People who retweeted x also retweeted y
print()
print("Sources that published the most retweets")
print("sid\tretweets")
for x, c in source_retweets.most_common(10):
    print(sid_name[x] + "\t" + str(c))
print()
print("Sources that retweeted the most unique targets")
print("sid\ttargets")
for x, c in source_target_count.most_common(10):
    print(sid_name[x] + "\t" + str(c))
    
communities = get_communities(interactions)
community_sids = {}
community_sizes = Counter()
for mod, names in communities.items():
    community_sizes[mod] = len(names)
print(len(communities))
print(community_sizes)

Number of sources: 25137
Number of targets: 8405
Total number of retweet interactions: 95893

Targets with most retweets
tid	retweets
user_025343	5477
user_000009	3480
user_020135	2350
user_001418	2323
user_001690	1708
user_000755	1100
user_004286	1067
user_025303	788
user_004000	750
user_000596	638

Targets with most unique sources retweeting them
tid	sources
user_025343	5477
user_000009	3480
user_020135	2350
user_001418	2323
user_001690	1708
user_000755	1100
user_004286	1067
user_025303	788
user_004000	750
user_000596	638

Sources that retweeted user_025343 also retweeted 873 other accounts.
Retweeted user_000009 195 times.
Retweeted user_001418 116 times.
Retweeted user_020135 105 times.
Retweeted user_000755 85 times.
Retweeted user_004286 62 times.
Retweeted user_000596 52 times.
Retweeted user_001690 52 times.
Retweeted user_000107 46 times.
Retweeted user_004000 30 times.
Retweeted user_000340 27 times.

Sources that retweeted user_000009 also retweeted 1992 other accounts.
Retw

## Choose accounts for poisoning experiment

In [ ]:
# Here a target to be boosted and a high-profile account that the target should be seen as similar are picked
# high_profile_tid was chosen from the original labeled dataset.
# It is a high-profile Twitter account that gets a lot of engagement
high_profile_tid = 191
high_profile_name = tid_name[high_profile_tid]
high_profile_sid = None
if high_profile_name in name_sid:
    high_profile_sid = name_sid[high_profile_name]
print("high_profile_tid: " + str(high_profile_tid) + " == " + tid_name[high_profile_tid])
print("high_profile_sid: " + str(high_profile_sid))
# The target account was selected based on a few criteria:
# - it is highly retweeted in the original dataset (top 10)
# - the original dataset contains plenty of accounts that haven't retweeted it and high_profile_tid
#   (thus enabling us to create a large number of amplifier candidates below)
target_tid = 4451
target_name = tid_name[target_tid]
target_sid = None
if target_name in name_sid:
    target_sid = name_sid[target_name]
print("target_tid: " + str(target_tid) + " == " + tid_name[target_tid])
print("target_sid: " + str(target_sid))
# Feel free to change these values for other experiments

# Pick a list of accounts that engaged with the high profile account in order to compare
# similarity values before and after poisoning
num_controls = 20
control_candidates = []
for sid, tids in source_retweeted.items():
    if len(tids) > 50:
        if high_profile_tid in tids:
            control_candidates.append(sid)
print("Candidates for control accounts: " + str(len(control_candidates)))
controls = random.sample(control_candidates, num_controls)
# For consistency's sake, here's a hard-coded list of control candidates 
# (selected by running the above code once)
# controls = [229, 6266, 340, 124, 25, 4000, 89, 4347, 1947, 20144, 14, 22, 107, 13426, 237, 708, 1560, 62, 9, 11]
print(controls)

high_profile_tid: 191 == user_025303
high_profile_sid: None
target_tid: 4451 == user_004286
target_sid: 4286
Candidates for control accounts: 53
[41, 1399, 6266, 14, 128, 25, 708, 408, 614, 1560, 280, 89, 22, 4286, 3, 1608, 350, 0, 4000, 130]


In [ ]:
# Find accounts that engaged with both high_profile and target
retweeted_target = target_retweeted_by[target_tid]
print("Number of accounts that retweeted target:")
print(len(retweeted_target))
retweeted_high_profile = target_retweeted_by[high_profile_tid]
print("Number of accounts that retweeted high-profile:")
print(len(retweeted_high_profile))
retweeted_both = set(retweeted_target).intersection(set(retweeted_high_profile))
print("Number of accounts that retweeted both:")
print(len(retweeted_both))

Number of accounts that retweeted target:
1067
Number of accounts that retweeted high-profile:
788
Number of accounts that retweeted both:
13


In [ ]:
# Feature analysis of communities 

retweeted_target = target_retweeted_by[target_tid]
retweeted_target_count = target_retweeted_count[target_tid]
retweeted_high_profile = target_retweeted_by[high_profile_tid]
retweeted_high_profile_count = target_retweeted_count[high_profile_tid]

G, N = make_nx_graph(interactions)
pr = nx.pagerank(G)

num_amplifiers = 200
community_features = {}
for mod, names in communities.items():
    amplifier_candidates = []
    for name in names:
        if name in name_sid:
            sid = name_sid[name]
            if sid in source_retweeted:
                rtw = source_retweeted[sid]
                if high_profile_tid not in rtw and target_tid not in rtw:
                    amplifier_candidates.append(sid)
    if len(amplifier_candidates) < num_amplifiers:
        continue
    print("Mod: " + str(mod) + " size: " + str(len(names)))
    community_features[mod] = {}
    community_features[mod]["Community size"] = len(names)
    
    community_total_retweets = 0
    for n in names:
        if n in name_sid:
            community_total_retweets += source_retweets[name_sid[n]]
    community_features[mod]["Total retweets"] = community_total_retweets

    mpr = get_mean(pr, names)
    community_features[mod]['Mean pagerank * 10e5'] = mpr * 100000

    jaccard = get_jaccard_mean(interactions, names, target_name)
    mod_name = 'Mean Jaccard coefficient between accounts in community and target * 10e4'
    community_features[mod][mod_name] = jaccard * 10000

    sid_list = [name_sid[x] for x in names if x in name_sid]
    rtw_target_sids = set(retweeted_target).intersection(set(sid_list))
    community_features[mod]['Unique accounts in community that retweeted target'] = len(rtw_target_sids)
    rtw_target_count = 0
    for sid, count in retweeted_target_count.items():
        if sid in rtw_target_sids:
            rtw_target_count += count
    community_features[mod]['Total retweets of target'] = rtw_target_count
    rhps = set(retweeted_high_profile).intersection(set(sid_list))
    community_features[mod]['Unique accounts in community that retweeted high-profile'] = len(rhps)
    rhpc = 0
    for sid, count in retweeted_high_profile_count.items():
        if sid in rhps:
            rhpc += count
    community_features[mod]['Total retweets of high-profile'] = rhpc
    community_retweet_counts = [source_retweets[x] for x in sid_list]
    community_features[mod]['Mean retweets per account'] = np.mean(community_retweet_counts)
    community_features[mod]['Max retweet count'] = max(community_retweet_counts)
    
    controls_in_mod = set(sid_list).intersection(set(controls))
    community_features[mod]['Number of control accounts'] = len(controls_in_mod)
        
    rtw_controls_sids = 0
    rtw_controls_count = 0
    for sid in controls:
        if sid in target_retweeted_count:
            rcl = target_retweeted_count[sid]
            for s, c in rcl.items():
                if s in sid_list:
                    rtw_controls_sids += 1
                    rtw_controls_count += c
    community_features[mod]['Accounts in this community that retweeted control accounts'] = rtw_controls_sids
    community_features[mod]['Total control account retweets published by this community'] = rtw_controls_count
    
    target_mean_path_len = get_mean_distance(interactions, target_name, names)
    community_features[mod]['Mean path length between community nodes and target'] = target_mean_path_len

print(json.dumps(community_features, indent=4))

Mod: 5 size: 1732
Mod: 4 size: 6788
Mod: 13 size: 2027
Mod: 6 size: 2128
Mod: 7 size: 2200
Mod: 8 size: 1790
Mod: 9 size: 368
Mod: 11 size: 1011
Mod: 0 size: 316
Mod: 14 size: 2832
Mod: 15 size: 2118
Mod: 16 size: 4731
Mod: 18 size: 1700
Mod: 19 size: 1403
Mod: 1 size: 263
{
    "5": {
        "Community size": 1732,
        "Total retweets": 6993,
        "Mean pagerank * 10e5": 3.3609910753994643,
        "Mean Jaccard coefficient between accounts in community and target * 10e4": 2.464483096763413,
        "Unique accounts in community that retweeted target": 25,
        "Total retweets of target": 30,
        "Unique accounts in community that retweeted high-profile": 17,
        "Total retweets of high-profile": 179,
        "Mean retweets per account": 12.4875,
        "Max retweet count": 3614,
        "Number of control accounts": 3,
        "Accounts in this community that retweeted control accounts": 16,
        "Total control account retweets published by this community": 41,

In [ ]:
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]
def highlight_min(s):
    is_min = s == s.min()
    return ['background-color: orange' if v else '' for v in is_min]
fe = pd.DataFrame(community_features)
fe.style.apply(highlight_max, axis=1).apply(highlight_min, axis=1).format("{:.2f}")

,5,4,13,6,7,8,9,11,0,14,15,16,18,19,1
Community size,1732.00,6788.00,2027.00,2128.00,2200.00,1790.00,368.00,1011.00,316.00,2832.00,2118.00,4731.00,1700.00,1403.00,263.00
Total retweets,6993.00,30847.00,8341.00,4002.00,2937.00,2903.00,916.00,2753.00,422.00,8948.00,7996.00,7462.00,5060.00,4509.00,300.00
Mean pagerank * 10e5,3.36,3.87,3.04,2.66,2.71,2.46,2.45,2.53,2.42,3.28,3.07,2.94,3.15,2.96,2.73
Mean Jaccard coefficient between accounts in community and target * 10e4,2.46,2.60,10.34,0.48,0.23,3.02,1.12,1.50,0.38,0.65,8.10,0.08,0.49,1.35,0.10
Unique accounts in community that retweeted target,25.00,45.00,810.00,5.00,12.00,8.00,2.00,16.00,0.00,17.00,87.00,6.00,3.00,26.00,1.00
Total retweets of target,30.00,45.00,1866.00,6.00,12.00,15.00,2.00,16.00,0.00,18.00,133.00,6.00,4.00,29.00,1.00
Unique accounts in community that retweeted high-profile,17.00,451.00,29.00,24.00,18.00,143.00,4.00,5.00,2.00,48.00,20.00,3.00,5.00,13.00,0.00
Total retweets of high-profile,179.00,1918.00,195.00,93.00,21.00,144.00,26.00,67.00,8.00,146.00,175.00,3.00,8.00,25.00,0.00
Mean retweets per account,12.49,10.43,4.95,1.91,1.35,1.66,3.74,4.07,1.51,3.19,4.78,1.58,2.99,3.35,1.14
Max retweet count,3614.00,3762.00,2230.00,316.00,99.00,14.00,453.00,1337.00,83.00,474.00,1559.00,47.00,200.00,839.00,4.00


## Build model

In [ ]:
# Given a ratings dataframe containing columns "Source", "Target", "Weight"
# train a collaborative filtering model and return the target and source weight embeds
def make_model_collab(ratings, epochs):
    min_rating = min(ratings["Weight"])
    max_rating = max(ratings["Weight"])
    print("Min rating: " + str(min_rating) + " Max rating: " + str(max_rating))
    dls = CollabDataLoaders.from_df(ratings, item_name='Target', bs=64)
    learn = collab_learner(dls, n_factors=50, y_range=(min_rating, max_rating))
    learn.fit_one_cycle(epochs)
    # Model weights
    target_w = learn.model.weight(dls.classes['Target'], is_item=True)
    source_w = learn.model.weight(dls.classes['Source'], is_item=False)
    return target_w, source_w

def make_model_nn(ratings, epochs):
    min_rating = min(ratings["Weight"])
    max_rating = max(ratings["Weight"])
    print("Min rating: " + str(min_rating) + " Max rating: " + str(max_rating))
    dls = CollabDataLoaders.from_df(ratings, item_name='Target', bs=64)
    learn = collab_learner(dls, use_nn=True, 
                           emb_szs={'userId': 50, 'movieId':50}, 
                           layers=[256, 128], y_range=(min_rating, max_rating))

    learn.fit_one_cycle(epochs)
    target_w = to_np(learn.model.embeds[1].weight[1:])
    source_w = to_np(learn.model.embeds[0].weight[1:])
    return target_w, source_w

def make_model(ratings, model_type, epochs):
    print("Model type: " + model_type)
    if model_type == "nn":
        return make_model_nn(ratings, epochs)
    else:
        return make_model_collab(ratings, epochs)

epochs = 5
model_type = "default"

In [ ]:
# Train collab model on base dataset
target_w, source_w = make_model(ratings, model_type, epochs)

print(target_w.shape)
print(source_w.shape)

# Calculate cosine similarity matrix between all targets in the set
t_matrix = cosine_similarity(target_w)
                            
print(t_matrix.shape)
print()

# Calculate cosine similarity matrix between all sources in the set
s_matrix = cosine_similarity(source_w)

print(s_matrix.shape)
print()

Model type: default
Min rating: 1 Max rating: 205


epoch,train_loss,valid_loss,time
0,8812.553711,8495.390625,00:15
1,4816.820312,4669.122070,00:17
2,2437.082520,3102.752197,00:17
3,1614.862305,2611.380371,00:15
4,1467.526245,2539.028076,00:13


torch.Size([7251, 50])
torch.Size([21071, 50])
(7251, 7251)

(21071, 21071)



## Recommendations by target similarity

In [ ]:
t_max_matches = 100 # top_n matches when doing target similarity

# Show closest matches to selected targets
samples = [target_tid, high_profile_tid]
print_similar_to_targets(samples, t_matrix)
print_target_similarity(target_tid, high_profile_tid, t_matrix)

Target: user_004286 similar to:
user_004286 1.0000
user_029017 0.9463
user_029316 0.9452
user_029150 0.9427
user_028743 0.9376
user_029228 0.9373
user_029816 0.9370
user_028817 0.9361
user_013935 0.9360
user_029312 0.9356

Target: user_025303 similar to:
user_025303 1.0000
user_025403 0.9370
user_025581 0.9361
user_025511 0.9351
user_025376 0.9341
user_026830 0.9320
user_025505 0.9313
user_025148 0.9290
user_025582 0.9282
user_025413 0.9279

user_004286 similarity to user_025303: 0.8578


In [ ]:
# graph tid similarity, to make a visualization
tid_inter = {}
threshold = 0.99
print(len(tid_name))
for tid, name in tid_name.items():
    matches = get_most_similar(tid, t_matrix, 50)
    if matches is not None:
        for item in matches:
            tid2, sim = item
            name2 = tid_name[tid2]
            if name != name2:
                if sim >= threshold:
                    if name not in tid_inter:
                        tid_inter[name] = Counter()
                    tid_inter[name][name2] = sim
print("Saving")
with open("tid_inter.csv", "w") as f:
    f.write("Source,Target,Weight\n")
    for source, targets in tid_inter.items():
        for target, weight in targets.items():
            f.write(str(source)+","+str(target)+","+str(weight)+"\n")
print("Done")

8405
Saving
Done


In [ ]:
# Build recommendations for source based on who they've retweeted
# For each target retweeted by a source, see if we have an entry in most_similar
# If we do, add each item to the recommended counter
# Assign the value to be the source's rating multiplied by the similarity score
# We'll also record what the user has already retweeted so we can recommend a target they haven't yet retweeted

# for target, num_retweets in get_source_retweets(source):
#    for similar, similarity in get_most_similar(target):
#        recommended[similar] += num_retweets * similarity

def get_user_recommendations_by_target(ratings, sid, t_matrix):
    s_ratings = ratings.loc[ratings['Source'] == sid]
    s_ratings = s_ratings.sort_values(by="Weight", ascending=False)
    s_r = list(zip(s_ratings['Target'], s_ratings['Weight']))
    recommended = Counter()
    seen = set()
    for item in s_r:
        tid, trating = item
        if tid > len(t_matrix):
            continue
        seen.add(tid)
        matches = get_most_similar(tid, t_matrix, t_max_matches)
        if matches != None:
            for entry in matches:
                t, r = entry
                recommended[t] += r * trating

    # Now we'll build a recomendations list that contains the highest scored items
    # calculated above that the user hasn't already rated
    seen_recommendations = Counter()
    not_seen_recommendations = Counter()
    for tid, score in recommended.most_common():
        if len(seen_recommendations) >= 10 and len(not_seen_recommendations) >= 10:
            break
        if tid not in seen:
            not_seen_recommendations[tid] = score
        else:
            seen_recommendations[tid] = score
    return seen_recommendations, not_seen_recommendations

def print_recommendations_by_target(sid, seen_recommendations, not_seen_recommendations):
    s_ratings = ratings.loc[ratings['Source'] == sid]
    s_ratings = s_ratings.sort_values(by="Weight", ascending=False)
    s_r = list(zip(s_ratings['Target'], s_ratings['Weight']))
    num_ratings = len(s_r)

    ind_rating = {}
    for item in s_r:
        ind, trating = item
        ind_rating[ind] = trating

    # Now let's print the output and see if it's sane
    print("User: " + sid_name[sid] + " retweeted " + str(num_ratings) + " different accounts.")
    print()
    top10 = []
    for item in s_r[:10]:
        tid, trating = item
        top10.append(tid)
        msg = "Retweeted by user: " + str(trating) + " times, total retweets: " + str(target_retweeters[tid]) 
        msg += "\t  " + tid_name[tid]
        print(msg)
    print()
    print("Recommended (seen):")
    for x, c in seen_recommendations.most_common(10):
        msg = "%.4f"%c + "\t(retweeted by user: " + str(ind_rating[x]) + " times,"
        msg += " total retweets: " + str(target_retweeters[x]) + ")" + "\t" + tid_name[x]
        if x == target_tid:
            msg += " [X]"
        if x in top10:
            msg += " [*]"
        print(msg)
    print()
    print("Recommended (not seen):")
    for x, c in not_seen_recommendations.most_common(10):
        msg = "%.4f"%c + "\t" + " (total retweets: " 
        msg += str(target_retweeters[x]) + ")\t" + tid_name[x]                  
        if x == target_tid:
            msg += " [X]"
        if x in top10:
            msg += " [*]"
        print(msg)
    print("=====================================================")
    print()

def print_user_recommendations_by_target(ratings, sid, t_matrix):
    seen, not_seen = get_user_recommendations_by_target(ratings, sid, t_matrix)
    print_recommendations_by_target(sid, seen, not_seen)

In [ ]:
# Display target-based recommendations for control set
for n in controls:
    print_user_recommendations_by_target(ratings, n, t_matrix)

User: user_000041 retweeted 171 different accounts.

Retweeted by user: 14 times, total retweets: 788	  user_025303
Retweeted by user: 4 times, total retweets: 2	  user_027283
Retweeted by user: 3 times, total retweets: 6	  user_004906
Retweeted by user: 3 times, total retweets: 101	  user_025218
Retweeted by user: 2 times, total retweets: 77	  user_025764
Retweeted by user: 2 times, total retweets: 43	  user_025266
Retweeted by user: 2 times, total retweets: 20	  user_024491
Retweeted by user: 2 times, total retweets: 82	  user_027313
Retweeted by user: 2 times, total retweets: 1	  user_027277
Retweeted by user: 2 times, total retweets: 126	  user_025171

Recommended (seen):
48.8365	(retweeted by user: 1 times, total retweets: 29)	user_025942
42.7630	(retweeted by user: 2 times, total retweets: 126)	user_025171 [*]
40.4249	(retweeted by user: 2 times, total retweets: 245)	user_025233
35.6384	(retweeted by user: 1 times, total retweets: 1)	user_009511
32.3609	(retweeted by user: 1 time

## Recommendations by source similarity

In [ ]:
s_max_matches = 100 # top_n matches when doing source similarity

# Print 10 closest sources for control set
print_similar_to_sources(controls, s_matrix)

User: user_000041 similar to:
user_000041 1.0000
user_000009 0.9965
user_001337 0.9951
user_001294 0.9941
user_000081 0.9940
user_001196 0.9940
user_001382 0.9935
user_000004 0.9926
user_000023 0.9922
user_000113 0.9921

User: user_001399 similar to:
user_001399 1.0000
user_002612 0.9282
user_010184 0.9250
user_008121 0.9249
user_007165 0.9222
user_020202 0.9214
user_020235 0.9211
user_003773 0.9195
user_018445 0.9193
user_007467 0.9189

User: user_006266 similar to:
user_006266 1.0000
user_012485 0.8891
user_010670 0.8844
user_006267 0.8771
user_014171 0.8707
user_003740 0.8653
user_000946 0.8622
user_004077 0.8565
user_014186 0.8555
user_009665 0.8504

User: user_000014 similar to:
user_000014 1.0000
user_000304 0.9957
user_001213 0.9945
user_001382 0.9943
user_001104 0.9938
user_001214 0.9922
user_000009 0.9920
user_001617 0.9918
user_002581 0.9910
user_000204 0.9907

User: user_000128 similar to:
user_000128 1.0000
user_014245 0.9254
user_010120 0.9219
user_002604 0.9141
user_01251

In [ ]:
# Recommendations based on source similarity
# From the previously calculated source similarities, calculate recommendations thus:
# For each similar source, obtain their list of target ratings
# Record a counter for each target where we add a value: similarity * rating
# Once we have a ranked list of recommendations, choose the top items
# based on whether the original user has rated the target or not

# for similar_source, similarity in get_most_similar(source):
#     for target, num_retweets in get_source_retweets(similar_source):
#         recommended[target] += similarity * num_retweets


def get_user_recommendations_by_source(ratings, sid, s_matrix):
    s_ratings = ratings.loc[ratings['Source'] == sid]
    s_ratings = s_ratings.sort_values(by="Weight", ascending=False)
    s_r = list(zip(s_ratings['Target'], s_ratings['Weight']))
    seen = set()
    tid_rating = {}
    for item in s_r:
        tid, trating = item
        tid_rating[tid] = trating
        seen.add(tid)

    recommended = Counter()
    matches = get_most_similar(sid, s_matrix, s_max_matches)
    if matches != None:
        for item in matches:
            sid, similarity = item
            ur = ratings.loc[ratings['Source'] == sid]
            ur = list(zip(ur['Target'], ur['Weight']))
            for entry in ur:
                tid, mr = entry
                recommended[tid] += similarity * mr

    # Now we'll build a recomendations list that contains the highest scored items
    # calculated above that the user hasn't already rated
    seen_recommendations = Counter()
    not_seen_recommendations = Counter()
    for tid, score in recommended.most_common():
        if len(seen_recommendations) >= 10 and len(not_seen_recommendations) >= 10:
            break
        if tid not in seen:
            not_seen_recommendations[tid] = score
        else:
            seen_recommendations[tid] = score
    return seen_recommendations, not_seen_recommendations

def print_recommendations_by_source(sid, seen_recommendations, not_seen_recommendations):
    s_ratings = ratings.loc[ratings['Source'] == sid]
    s_ratings = s_ratings.sort_values(by="Weight", ascending=False)
    s_r = list(zip(s_ratings['Target'], s_ratings['Weight']))
    num_ratings = len(s_r)

    tid_rating = {}
    for item in s_r:
        tid, mrating = item
        tid_rating[tid] = mrating

    # Now let's print the output and see if it's sane
    print("User: " + sid_name[sid] + " retweeted " + str(num_ratings) + " different accounts.")
    print()
    top10 = []
    for item in s_r[:10]:
        tid, trating = item
        top10.append(tid)
        msg = "Retweeted by user: " + str(trating) + " times, total retweets: " + str(target_retweeters[tid]) 
        msg += "\t  " + tid_name[tid]
        print(msg)
    print()
    print("Recommended (seen):")
    for x, c in seen_recommendations.most_common(10):
        msg = "%.4f"%c
        msg += " (retweeted by user: " + str(tid_rating[x]) + " times,"
        msg += " total retweets: " + str(target_retweeters[x]) + ")" + "\t" 
        msg += tid_name[x]
        if x == target_tid:
            msg += " [X]"
        if x in top10:
            msg += " [*]"
        print(msg)
    print()
    print("Recommended (not seen):")
    for x, c in not_seen_recommendations.most_common(10):
        msg = "%.4f"%c 
        msg += " (total retweets: " + str(target_retweeters[x]) + ")\t"
        msg += tid_name[x]
        if x == target_tid:
            msg += " [X]"
        if x in top10:
            msg += " [*]"
        print(msg)
    print("=====================================================")
    print()

def print_user_recommendations_by_source(ratings, sid, s_matrix):
    seen, not_seen = get_user_recommendations_by_source(ratings, sid, s_matrix)
    print_recommendations_by_source(sid, seen, not_seen)

In [ ]:
# Print source-based recommendations for control set
for n in controls:
    print_user_recommendations_by_source(ratings, n, s_matrix)

User: user_000041 retweeted 171 different accounts.

Retweeted by user: 14 times, total retweets: 788	  user_025303
Retweeted by user: 4 times, total retweets: 2	  user_027283
Retweeted by user: 3 times, total retweets: 6	  user_004906
Retweeted by user: 3 times, total retweets: 101	  user_025218
Retweeted by user: 2 times, total retweets: 77	  user_025764
Retweeted by user: 2 times, total retweets: 43	  user_025266
Retweeted by user: 2 times, total retweets: 20	  user_024491
Retweeted by user: 2 times, total retweets: 82	  user_027313
Retweeted by user: 2 times, total retweets: 1	  user_027277
Retweeted by user: 2 times, total retweets: 126	  user_025171

Recommended (seen):
392.8223 (retweeted by user: 14 times, total retweets: 788)	user_025303 [*]
182.9197 (retweeted by user: 2 times, total retweets: 245)	user_025233
128.1706 (retweeted by user: 3 times, total retweets: 101)	user_025218 [*]
103.9492 (retweeted by user: 1 times, total retweets: 3480)	user_000009
83.8019 (retweeted by

## Validation: count how many times target appeared in top_n recommendations for each source

In [ ]:
def validate_target_target(ratings, sid, t_matrix, target, top_n):
    ret = False
    seen, not_seen = get_user_recommendations_by_target(ratings, sid, t_matrix)
    top_tids = [x for x, c in not_seen.most_common(top_n)]
    if target in top_tids:
        ret = True
    return ret

def validate_target_source(ratings, sid, s_matrix, target, top_n):
    ret = False
    seen, not_seen = get_user_recommendations_by_source(ratings, sid, s_matrix)
    top_tids = [x for x, c in not_seen.most_common(top_n)]
    if target in top_tids:
        ret = True
    return ret

In [ ]:
ret = 0
top_n = 3
for sid in controls:
    found = validate_target_target(ratings, sid, t_matrix, target_tid, top_n)
    if found == True:
        ret += 1
print("Target was in top "+str(top_n)+" target-based recommendations for "+str(ret)+" users in control list.")

Target was in top 3 target-based recommendations for 0 users in control list.


In [ ]:
res = 0
top_n = 3
for sid in controls:
    found = validate_target_source(ratings, sid, s_matrix, target_tid, top_n)
    if found == True:
        res += 1
print("Target was in top "+str(top_n)+" source-based recommendations for "+str(res)+" users in control list.")

Target was in top 3 source-based recommendations for 1 users in control list.


In [ ]:
# STOP

## Poisoning experiment 1 - randomly chosen amplifiers, variable amps, retweets
- with differing numbers of amplifiers and retweets:
    - repeat "iterations" times
        - create new poisoned dataframe based on supplied parameters
        - save csv for gephi visualization
        - train model
        - run source-based and target-based recommendations, see how often target appears in top_n recommendations
        - record all results to be graphed later

In [ ]:
# Run poisoning experiment 1
# Note that this cell can take a number of hours to run

# These are the experiments to run
# Each pair of numbers denotes [num_amplifiers, num_retweets]
# A poisoned copy of the dataset is generated as follows:
# 1. Make a copy of the original dataset
# 2. Randomly select num_amplifiers accounts from amplifier candidates
# 3. For each selected amplifier, add two rows to the copied dataset:
#    amplifier - target - num_retweets
#    amplifier - high_profile_user - num_retweets
# 4. Perform the rest of the experiment (train model, analyze recomendations)
experiments = [[0,0],
               [10,10], [10,20], [10,50], [10,100],
               [20,10], [20,20], [20,50], [20,100],
               [50,10], [50,20], [50,50], [50,100],
               [100,1], [100,5], [100,10], [100,20], [100,50],
               [200,1], [200,5], [200,10], [200,20], [200,50],
               [500,1], [500,5], [500,10], [500,20], [500,50],
               [1000,1], [1000,5], [1000,10], [1000,20], [1000,50],
               [2000,1], [2000,5], [2000,10], [2000,20], [2000,50],
               [4000,1], [4000,5], [4000,10], [4000,20], [4000,50]]

samples = [target_tid, high_profile_tid]
top_n = 3
iterations = 10
epochs = 5

# 1. Pick random accounts (not in control set) to do the boosting 
# that havent engaged with either high profile or target
amplifier_candidates = []
for sid, tids in source_retweeted.items():
    if len(tids) > 0:
        inter = set(tids).intersection(set(controls))
        if len(inter) == 0:
            if high_profile_tid not in tids and target_tid not in tids:
                amplifier_candidates.append(sid)
print("Number of random amplifier candidates: " + str(len(amplifier_candidates)))

# Loop through the experiment parameters
# For each set of parameters, perform the experiment iterations number of times
result_source = []
result_target = []
i = 1
save_dir = "US2020/exp1"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
for item in experiments:
    amps, r = item
    for n in range(iterations):
        print()
        print("Experiment:" + str(i) + " amps:" + str(amps) + " r:" + str(r) + " take:" + str(n))
        i = i + 1
        save_path = save_dir + "/" + str(amps) + "_" + str(r) + "_" + str(n) + ".csv"
        new_ratings = get_poisoned_dataset(ratings, amplifier_candidates, amps, r, save_path)
        msg = "Base dataset length: " + str(len(ratings))
        msg += " Poisoned dataset length: " + str(len(new_ratings))
        print(msg)
        new_target_w, new_source_w = make_model(new_ratings, model_type, epochs)
        new_t_matrix = cosine_similarity(new_target_w)
        print_target_similarity(target_tid, high_profile_tid, new_t_matrix)
        new_s_matrix = cosine_similarity(new_source_w)
        ret = 0
        for sid in controls:
            found = validate_target_target(new_ratings, sid, new_t_matrix, target_tid, top_n)
            if found == True:
                ret += 1
        msg = "Target was in top " + str(top_n) 
        msg += " target recommendations for " + str(ret) 
        msg += " users in control list."
        print(msg)
        result_target.append([amps, r, n, ret])
        with open(save_dir + "/result_target.json", "w") as f:
            f.write(json.dumps(result_target, indent=4))
        res = 0
        for sid in controls:
            found = validate_target_source(new_ratings, sid, new_s_matrix, target_tid, top_n)
            if found == True:
                res += 1
        msg = "Target was in top " + str(top_n) 
        msg += " source recommendations for " + str(res) 
        msg += " users in control list."
        print(msg)
        result_source.append([amps, r, n, res])
        with open(save_dir + "/result_source.json", "w") as f:
            f.write(json.dumps(result_source, indent=4))

Number of random amplifier candidates: 23256

Experiment:1 amps:0 r:0 take:0
Base dataset length: 52920 Poisoned dataset length: 52920
Model type: default
Min rating: 1 Max rating: 205


epoch,train_loss,valid_loss,time
0,8794.023438,8474.701172,00:15
1,4793.515137,4715.096680,00:17
2,2482.211670,3186.336670,00:18
3,1646.312012,2704.375244,00:16
4,1493.106689,2633.339844,00:13


user_004286 similarity to user_025303: 0.8395
Target was in top 3 target recommendations for 0 users in control list.
Target was in top 3 source recommendations for 0 users in control list.

Experiment:2 amps:0 r:0 take:1
Base dataset length: 52920 Poisoned dataset length: 52920
Model type: default
Min rating: 1 Max rating: 205


epoch,train_loss,valid_loss,time
0,8797.510742,8507.350586,00:16
1,4780.299316,4720.615723,00:17
2,2479.473877,3168.660645,00:18
3,1572.855591,2682.748535,00:16
4,1452.523438,2611.063965,00:14


user_004286 similarity to user_025303: 0.8706
Target was in top 3 target recommendations for 0 users in control list.
Target was in top 3 source recommendations for 2 users in control list.

Experiment:3 amps:0 r:0 take:2
Base dataset length: 52920 Poisoned dataset length: 52920
Model type: default
Min rating: 1 Max rating: 205


...

epoch,train_loss,valid_loss,time
0,8822.141602,8529.401367,00:17
1,4789.171875,4719.748047,00:18
2,2460.475098,3168.352783,00:18
3,1637.836304,2683.716797,00:17
4,1415.376343,2612.457031,00:14


user_004286 similarity to user_025303: 0.8924
Target was in top 3 target recommendations for 0 users in control list.
Target was in top 3 source recommendations for 2 users in control list.

Experiment:106 amps:50 r:20 take:5
Base dataset length: 52920 Poisoned dataset length: 53020
Model type: default
Min rating: 1 Max rating: 205


epoch,train_loss,valid_loss,time
0,8815.672852,8547.629883,00:16
1,4758.927734,4723.821777,00:18
2,2409.365967,3181.205322,00:18
3,1607.258789,2701.637939,00:16
4,1419.175049,2630.638672,00:14


user_004286 similarity to user_025303: 0.7818
Target was in top 3 target recommendations for 0 users in control list.
Target was in top 3 source recommendations for 4 users in control list.

Experiment:107 amps:50 r:20 take:6
Base dataset length: 52920 Poisoned dataset length: 53020
Model type: default
Min rating: 1 Max rating: 205


epoch,train_loss,valid_loss,time


## Display results as a plot

In [ ]:
filename = "exp1/result_source.json"
title = "US2020 Experiment 1 - source-based recommendations"
results = []
with open(filename, "r") as f:
    results = json.loads(f.read())
results2 = []
order = []
for item in results:
    if len(item) == 4:
        a, r, t, v = item
        l = str(a) + "_" + str(r)
    else:
        m, t, v = item
        l = str(m)
    v = (v/20)*100
    if l not in order:
        order.append(l)
    results2.append([l, t, v])
df = pd.DataFrame(results2, columns=["params", "take", "val"])

plt.figure()
ax = None
fig = plt.figure(figsize=(20,8))
sns.set(style="whitegrid")
ax = sns.barplot(x="params", y="val", data=df, order=order, capsize=.2)
ax.set_title(title)
xlab = "Experiment parameters (num_accounts, num_retweets)"
ylab = "Percentage of control set that saw target account in top-3 recommendations"
plt.xlabel(xlab)
plt.ylabel(ylab)
for item in ax.get_xticklabels():
    item.set_rotation(45)

In [ ]:
filename = "exp1/result_target.json"
title = "US2020 Experiment 1 - target-based recommendations"
results = []
with open(filename, "r") as f:
    results = json.loads(f.read())
results2 = []
order = []
for item in results:
    if len(item) == 4:
        a, r, t, v = item
        l = str(a) + "_" + str(r)
    else:
        m, t, v = item
        l = str(m)
    v = (v/20)*100
    if l not in order:
        order.append(l)
    results2.append([l, t, v])
df = pd.DataFrame(results2, columns=["params", "take", "val"])

plt.figure()
ax = None
fig = plt.figure(figsize=(20,8))
sns.set(style="whitegrid")
ax = sns.barplot(x="params", y="val", data=df, order=order, capsize=.2)
ax.set_title(title)
xlab = "Experiment parameters (num_accounts, num_retweets)"
ylab = "Percentage of control set that saw target account in top-3 recommendations"
plt.xlabel(xlab)
plt.ylabel(ylab)
for item in ax.get_xticklabels():
    item.set_rotation(45)

In [ ]:
STOP

## Poisoning experiment 2 - amplifiers chosen based on community
- with fixed number of amplifiers and retweets:
    - iterate through communities (discovered from louvain method)
    - if a community contains at least num_amplifiers, select a set of amplifiers randomly from the community
    - repeat "iterations" times
        - create new poisoned dataframe based on supplied parameters
        - train model
        - run source-based and target-based recommendations, see how often target appears in top_n recommendations
        - record all results to be graphed later

In [ ]:
# Run poisoning experiment 2

num_amplifiers = 200
num_retweets = 20

samples = [target_tid, high_profile_tid]
top_n = 3
iterations = 10
epochs = 5

result_source = []
result_target = []
i = 1
save_dir = "exp2"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

target_name = tid_name[target_tid]
high_profile_name = tid_name[high_profile_tid]
print("Community labels: " + ", ".join([str(x) for x in communities.keys()]))
for mod, names in communities.items():
    if target_name in names:
        print("target: " + target_name + " in community: " + str(mod) + " size: " + str(len(names)))
    if high_profile_name in names:
        print("high_profile: " + high_profile_name + " in community: " + str(mod) + " size: " + str(len(names)))

for mod, names in sorted(communities.items()):
    print("Community: " + str(mod) + " contains " + str(len(names)) + " names.")
    amplifier_candidates = []
    for name in names:
        if name in name_sid:
            sid = name_sid[name]
            if sid in source_retweeted:
                rtw = source_retweeted[sid]
                if high_profile_tid not in rtw and target_tid not in rtw:
                    amplifier_candidates.append(sid)
    if len(amplifier_candidates) < num_amplifiers:
        print("Skipping community: "+str(mod)+" (only found "+str(len(amplifier_candidates))+" candidates).")
        continue
    for n in range(iterations):
        print()
        print("Experiment:" + str(i) + " community:" + str(mod) + " take:" + str(n))
        i = i + 1
        save_path = save_dir + "/" + str(mod) + "_" + str(n) + ".csv"
        new_ratings = get_poisoned_dataset(ratings, amplifier_candidates, 
                                           num_amplifiers, num_retweets, save_path)
        msg = "Base dataset length: " + str(len(ratings))
        msg += " Poisoned dataset length: " + str(len(new_ratings))
        print(msg)
        new_target_w, new_source_w = make_model(new_ratings, model_type, epochs)
        new_t_matrix = cosine_similarity(new_target_w)
        print_target_similarity(target_tid, high_profile_tid, new_t_matrix)
        new_s_matrix = cosine_similarity(new_source_w)
        ret = 0
        for sid in controls:
            found = validate_target_target(new_ratings, sid, new_t_matrix, target_tid, top_n)
            if found == True:
                ret += 1
        msg = "Target was in top " + str(top_n) 
        msg += " target recommendations for " + str(ret) 
        msg += " users in control list."
        print(msg)
        result_target.append([mod, n, ret])
        with open(save_dir + "/result_target.json", "w") as f:
            f.write(json.dumps(result_target, indent=4))
        res = 0
        for sid in controls:
            found = validate_target_source(new_ratings, sid, new_s_matrix, target_tid, top_n)
            if found == True:
                res += 1
        msg = "Target was in top " + str(top_n) 
        msg += " source recommendations for " + str(res) 
        msg += " users in control list."
        print(msg)
        result_source.append([mod, n, res])
        with open(save_dir + "/result_source.json", "w") as f:
            f.write(json.dumps(result_source, indent=4))

In [ ]:
filename = "exp2/result_source.json"
title = "US2020 Experiment 2 - source-based recommendations"
results = []
with open(filename, "r") as f:
    results = json.loads(f.read())
results2 = []
order = []
for item in results:
    if len(item) == 4:
        a, r, t, v = item
        l = str(a) + "_" + str(r)
    else:
        m, t, v = item
        l = str(m)
    v = (v/20)*100
    if l not in order:
        order.append(l)
    results2.append([l, t, v])
df = pd.DataFrame(results2, columns=["params", "take", "val"])

plt.figure()
ax = None
fig = plt.figure(figsize=(20,8))
sns.set(style="whitegrid")
ax = sns.barplot(x="params", y="val", data=df, order=order, capsize=.2)
ax.set_title(title)
xlab = "Experiment parameters (community label)"
ylab = "Percentage of control set that saw target account in top-3 recommendations"
plt.xlabel(xlab)
plt.ylabel(ylab)
for item in ax.get_xticklabels():
    item.set_rotation(45)

In [ ]:
filename = "exp2/result_target.json"
title = "US2020 Experiment 2 - target-based recommendations"
results = []
with open(filename, "r") as f:
    results = json.loads(f.read())
results2 = []
order = []
for item in results:
    if len(item) == 4:
        a, r, t, v = item
        l = str(a) + "_" + str(r)
    else:
        m, t, v = item
        l = str(m)
    v = (v/20)*100
    if l not in order:
        order.append(l)
    results2.append([l, t, v])
df = pd.DataFrame(results2, columns=["params", "take", "val"])

plt.figure()
ax = None
fig = plt.figure(figsize=(20,8))
sns.set(style="whitegrid")
ax = sns.barplot(x="params", y="val", data=df, order=order, capsize=.2)
ax.set_title(title)
xlab = "Experiment parameters (community label)"
ylab = "Percentage of control set that saw target account in top-3 recommendations"
plt.xlabel(xlab)
plt.ylabel(ylab)
for item in ax.get_xticklabels():
    item.set_rotation(45)

In [ ]:
STOP

## Poisoning experiment 3 - amplifiers chosen based on similarity to control accounts
- with varying number of amplifiers and retweets:
    - select a set of amplifiers that are similar to control accounts
    - repeat "iterations" times
        - create new poisoned dataframe based on supplied parameters
        - train model
        - run source-based and target-based recommendations, see how often target appears in top_n recommendations
        - record all results to be graphed later

In [ ]:
# Run poisoning experiment 3

experiments = [[0,0],
               [100,1], [100,5], [100,10], [100,20],
               [200,1], [200,5], [200,10], [200,20], 
               [500,1], [500,5], [500,10], [500,20], 
               [1000,1], [1000,5], [1000,10], [1000,20],
               [2000,1], [2000,5], [2000,10], [2000,20]]

samples = [target_tid, high_profile_tid]
top_n = 3
iterations = 10
epochs = 5

# 1. Pick accounts most similar to those in the control set
# that havent engaged with either high profile or target
# and aren't in the control group
amplifier_candidates = []
sims = set()
for sid in controls:
    sim = get_most_similar(sid, s_matrix, 250)
    for s, _ in sim:
        if s not in controls:
            if s in source_retweeted:
                rtw = source_retweeted[s]
                if high_profile_tid not in rtw and target_tid not in rtw:
                    sims.add(s)
amplifier_candidates = list(sims)
print("Number of amplifier candidates: " + str(len(amplifier_candidates)))
# Loop through the experiment parameters
# For each set of parameters, perform the experiment iterations number of times
result_source = []
result_target = []
i = 1
save_dir = "exp3"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
for item in experiments:
    amps, r = item
    for n in range(iterations):
        print()
        print("Experiment:" + str(i) + " amps:" + str(amps) + " r:" + str(r) + " take:" + str(n))
        i = i + 1
        save_path = save_dir + "/" + str(amps) + "_" + str(r) + "_" + str(n) + ".csv"
        new_ratings = get_poisoned_dataset(ratings, amplifier_candidates, amps, r, save_path)
        msg = "Base dataset length: " + str(len(ratings))
        msg += " Poisoned dataset length: " + str(len(new_ratings))
        print(msg)
        new_target_w, new_source_w = make_model(new_ratings, model_type, epochs)
        new_t_matrix = cosine_similarity(new_target_w)
        print_target_similarity(target_tid, high_profile_tid, new_t_matrix)
        new_s_matrix = cosine_similarity(new_source_w)
        ret = 0
        for sid in controls:
            found = validate_target_target(new_ratings, sid, new_t_matrix, target_tid, top_n)
            if found == True:
                ret += 1
        msg = "Target was in top " + str(top_n) 
        msg += " target recommendations for " + str(ret) 
        msg += " users in control list."
        print(msg)
        result_target.append([amps, r, n, ret])
        with open(save_dir + "/result_target.json", "w") as f:
            f.write(json.dumps(result_target, indent=4))
        res = 0
        for sid in controls:
            found = validate_target_source(new_ratings, sid, new_s_matrix, target_tid, top_n)
            if found == True:
                res += 1
        msg = "Target was in top " + str(top_n) 
        msg += " source recommendations for " + str(res) 
        msg += " users in control list."
        print(msg)
        result_source.append([amps, r, n, res])
        with open(save_dir + "/result_source.json", "w") as f:
            f.write(json.dumps(result_source, indent=4))

In [ ]:
filename = "exp3/result_source.json"
title = "US2020 Experiment 3 - source-based recommendations"
results = []
with open(filename, "r") as f:
    results = json.loads(f.read())
results2 = []
order = []
for item in results:
    if len(item) == 4:
        a, r, t, v = item
        l = str(a) + "_" + str(r)
    else:
        m, t, v = item
        l = str(m)
    v = (v/20)*100
    if l not in order:
        order.append(l)
    results2.append([l, t, v])
df = pd.DataFrame(results2, columns=["params", "take", "val"])

plt.figure()
ax = None
fig = plt.figure(figsize=(20,8))
sns.set(style="whitegrid")
ax = sns.barplot(x="params", y="val", data=df, order=order, capsize=.2)
ax.set_title(title)
xlab = "Experiment parameters (num_accounts, num_retweets)"
ylab = "Percentage of control set that saw target account in top-3 recommendations"
plt.xlabel(xlab)
plt.ylabel(ylab)
for item in ax.get_xticklabels():
    item.set_rotation(45)

In [ ]:
filename = "exp3/result_target.json"
title = "US2020 Experiment 3 - target-based recommendations"
results = []
with open(filename, "r") as f:
    results = json.loads(f.read())
results2 = []
order = []
for item in results:
    if len(item) == 4:
        a, r, t, v = item
        l = str(a) + "_" + str(r)
    else:
        m, t, v = item
        l = str(m)
    v = (v/20)*100
    if l not in order:
        order.append(l)
    results2.append([l, t, v])
df = pd.DataFrame(results2, columns=["params", "take", "val"])

plt.figure()
ax = None
fig = plt.figure(figsize=(20,8))
sns.set(style="whitegrid")
ax = sns.barplot(x="params", y="val", data=df, order=order, capsize=.2)
ax.set_title(title)
xlab = "Experiment parameters (num_accounts, num_retweets)"
ylab = "Percentage of control set that saw target account in top-3 recommendations"
plt.xlabel(xlab)
plt.ylabel(ylab)
for item in ax.get_xticklabels():
    item.set_rotation(45)

In [ ]:
STOP